In [1]:
import pandas as pd
import numpy as np
from chembl_webresource_client.new_client import new_client

In [3]:
columns_rf = ["ChEMBL ID","Frequency","Avg Predicted Value"]
GRF = pd.read_csv("..\RF gtpase code\\25.5.0_RF_molecules_Newfeatures_GTPase_KRas.csv")
CRF = pd.read_csv("..\RF G12C Code\\25.5.7_RF_molecules_Newfeatures_G12C_trial.csv")
DRF = pd.read_csv("..\RF G12D code\\25.5.7_Final_molecules_RF_G12D.csv", names=columns_rf)
GNN = pd.read_csv("..\\NN GTPase Code\\25.5.0_NN_molecules_Newfeatures_GTPase_KRAS.csv")
CNN = pd.read_csv("..\\NN G12C Code\\25.5.7_NN_molecules_Newfeatures_G12C.csv")
DNN = pd.read_csv("..\\NN G12D Code\\25.5.5_NN_molecules_Newfeatures_G12D_new.csv")

In [4]:
# use this to convert the Neural Network pIC50 values to floats in the dataframe
GNN["Predicted_Value"] = GNN["Predicted_Value"].str.extract(r'np\.float32\((.*?)\)')[0].astype(float)
CNN["Predicted_Value"] = CNN["Predicted_Value"].str.extract(r'np\.float32\((.*?)\)')[0].astype(float)
DNN["Predicted_Value"] = DNN["Predicted_Value"].str.extract(r'np\.float32\((.*?)\)')[0].astype(float)
#DNN["Predicted_Value"] = DNN["Predicted_Value"].str.extract(r'\[(.*?)\]')[0].astype(float)


# and also drop the Unnamed: 0 column
GNN = GNN.drop(["Unnamed: 0"], axis=1)
CNN = CNN.drop(["Unnamed: 0"], axis=1)
DNN = DNN.drop(["Unnamed: 0"], axis=1)
CNN.head()

,chembl_id,Predicted_Value
0,CHEMBL3301603,8.891777
1,CHEMBL1088977,7.537722
2,CHEMBL1750,7.266871
3,CHEMBL2218878,7.188524
4,CHEMBL3348963,7.135389


In [5]:
# drop column frequency for Random Forest
GRF = GRF.drop(["Frequency"], axis=1)
DRF = DRF.drop(["Frequency"], axis=1)
CRF = CRF.drop(["Frequency"], axis=1)

# rename them to fit Neural Network convention
GRF.rename(columns={"ChEMBL ID": "chembl_id", "Avg Predicted Value": "Predicted_Value"}, inplace=True)
DRF.rename(columns={"ChEMBL ID": "chembl_id", "Avg Predicted Value": "Predicted_Value"}, inplace=True)
CRF.rename(columns={"ChEMBL ID": "chembl_id", "Avg Predicted Value": "Predicted_Value"}, inplace=True)
GRF.head()

,chembl_id,Predicted_Value
0,CHEMBL4594350,8.106427
1,CHEMBL2359966,7.497036
2,CHEMBL2361370,7.419709
3,CHEMBL3989958,7.372708
4,CHEMBL3707183,7.497036


In [6]:
df_set = {
  "Gtpase Random Forest": GRF,
  "G12C Random Forest": CRF,
  "G12D Random Forest": DRF,
  "Gtpase Neural Network": GNN,
  "G12C Neural Network": CNN,
  "G12D Neural Network": DNN
}

In [7]:
list = []
for key, value in df_set.items():
    list.append([key, " "," "])
    df = value
    for i in df["chembl_id"]:
        molecule = new_client.molecule
        m1 = molecule.filter(chembl_id=i).only(
            ['molecule_chembl_id', 'pref_name', 'molecule_structures'])
        targets = pd.DataFrame.from_dict(m1)
        #print(targets['molecule_chembl_id'][0])
        if targets['molecule_chembl_id'][0] == i:
            list.append([i,targets['pref_name'][0],float(df.loc[df["chembl_id"]== i ]["Predicted_Value"].values[0])])
            # print(i,targets['pref_name'][0],df[df["chembl_id"]== i ]["Predicted_Value"].values[0])
    list.append([" ", " "," "])

In [8]:
list

[['Gtpase Random Forest', ' ', ' '],
 ['CHEMBL4594350', 'ADAGRASIB', 8.106427136975098],
 ['CHEMBL2359966', 'QUININE SULFATE', 7.497035616770629],
 ['CHEMBL2361370', 'PENBUTOLOL SULFATE', 7.419709268156769],
 ['CHEMBL3989958', 'IVOSIDENIB', 7.372708472127514],
 ['CHEMBL3707183', 'QUINIDINE SULFATE', 7.497035616770629],
 ['CHEMBL4297091', 'CODEINE SULFATE', 7.425687232920642],
 ['CHEMBL3187985', 'APOMORPHINE HYDROCHLORIDE', 7.257022141325173],
 ['CHEMBL2103744', 'MORPHINE SULFATE', 7.2252363553078975],
 ['CHEMBL3183409', 'APALUTAMIDE', 7.217008092870295],
 ['CHEMBL4535757', 'SOTORASIB', 7.173295985258044],
 [' ', ' ', ' '],
 ['G12C Random Forest', ' ', ' '],
 ['CHEMBL186', 'CEFEPIME', 7.560583672313528],
 ['CHEMBL1200962', 'CEFEPIME HYDROCHLORIDE', 7.563626799697784],
 ['CHEMBL1201163', 'SUFENTANIL CITRATE', 7.17339856304644],
 ['CHEMBL3187246', 'METHAPYRILENE FUMARATE', 7.259904943281607],
 ['CHEMBL658', 'SUFENTANIL', 7.0408322787311945],
 ['CHEMBL2105458', 'THENALIDINE', 7.03910646712

In [9]:
final = pd.DataFrame(list)

In [10]:
final.head()

,0,1,2
0,Gtpase Random Forest,,
1,CHEMBL4594350,ADAGRASIB,8.106427
2,CHEMBL2359966,QUININE SULFATE,7.497036
3,CHEMBL2361370,PENBUTOLOL SULFATE,7.419709
4,CHEMBL3989958,IVOSIDENIB,7.372708


In [11]:
import datetime
final.to_excel("{}full_mol_list.xlsx".format(datetime.datetime.now().strftime("%y.%m.%d_")) , index=False)
